In [1]:
import pickle
import pandas as pd
import torch
import torch.nn as nn
import numpy as np

from utils import seed_everything, set_device, get_train_val_dataloaders, train
from model import FeedForwardRegressionNet

from torch.utils.data import TensorDataset
from torch.optim import Adam

In [2]:
n_jobs = 1000
results = []

for i in range(n_jobs):
    results.append(pickle.load(open(f"results/training_stats_{i}", "rb")))

In [3]:
df = pd.DataFrame(results).sort_values("Valid. Loss")

In [4]:
df[0:10]

,Training Loss,Valid. Loss,Valid. CI,Training Time,Validation Time,hidden_dims,learning_rate,embed_dim0,embed_dim1,epochs,job_id,weight_decay
909,5949.000052,7202.642291,853.347649,0:00:03,0:00:00,"[128, 8, 64, 16]",0.003,16,16,32,909,0.0000
643,7362.077195,7805.821926,1135.308080,0:00:02,0:00:00,"[128, 4, 8, 64]",0.003,4,16,32,643,0.0300
637,7467.503731,8533.302768,1241.257811,0:00:02,0:00:00,"[128, 8, 16, 64]",0.010,64,16,32,637,0.0010
230,10289.032137,8672.418840,1064.037984,0:00:02,0:00:00,"[128, 64]",0.010,16,32,16,230,0.0030
941,8778.780536,8890.790843,920.127426,0:00:03,0:00:00,"[128, 64]",0.010,32,16,16,941,0.0030
980,8960.864510,9274.521630,1342.891781,0:00:03,0:00:00,"[128, 64, 16]",0.003,8,16,16,980,0.0100
244,8972.532534,9697.145794,1492.363828,0:00:03,0:00:00,"[16, 128, 4]",0.003,4,32,32,244,0.0003
183,11016.088063,10072.006963,1290.892686,0:00:02,0:00:00,"[128, 64, 4, 16]",0.003,4,4,16,183,0.0300
490,9616.880451,10138.400236,1540.228110,0:00:02,0:00:00,"[128, 8, 64]",0.003,16,8,16,490,0.0010
892,12379.667142,10738.246195,1333.799068,0:00:03,0:00:00,"[64, 16, 4]",0.010,8,4,16,892,0.0003


In [13]:
best_hyperparameters = {
    "hidden_dims": [128, 64],
    "learning_rate": 0.01,
    "embed_dim0": 4,
    "embed_dim1": 8,
    "epochs": 32,
    "weight_decay" : 0.03
}

pickle.dump(best_hyperparameters, open("best_hyperparameters.pkl", "wb"))

In [27]:
hyperparameters = pickle.load(open("best_hyperparameters.pkl", "rb"))

# hyperparameters = {
#     "hidden_dims": [128, 64],
#     "learning_rate": 0.01,
#     "embed_dim0": 4,
#     "embed_dim1": 8,
#     "epochs": 32,
#     "weight_decay" : 0.03
# }

HIDDEN_DIMS = hyperparameters["hidden_dims"]
LEARNING_RATE = hyperparameters["learning_rate"]
EMBED_DIM0 = hyperparameters["embed_dim0"]
EMBED_DIM1 = hyperparameters["embed_dim1"]
EPOCHS = hyperparameters["epochs"]

batch_size = 64
val_size = 0.1

In [28]:
device = set_device()

X = pickle.load(open("preprocessed_data/X_train_tensor.pkl", "rb"))
y = pickle.load(open("preprocessed_data/y_train_tensor.pkl", "rb"))

dataset = TensorDataset(X, y)

train_dataloader, validation_dataloader = get_train_val_dataloaders(
    dataset, val_size, batch_size
)

No GPU available, using the CPU instead.
45,000 training samples
4,999 validation samples


In [29]:
input_dim = X.shape[1] - 2
vocab_size0 = len(np.unique(X[:, 0]))
vocab_size1 = len(np.unique(X[:, 1]))
output_dim = 1

model = FeedForwardRegressionNet(
    input_dim,
    vocab_size0,
    vocab_size1,
    EMBED_DIM0,
    EMBED_DIM1,
    HIDDEN_DIMS,
    output_dim,
)

In [30]:
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)

criterion = nn.MSELoss()

seed_everything(42)

training_stats = train(
    model,
    EPOCHS,
    train_dataloader,
    validation_dataloader,
    criterion,
    optimizer,
    device=device,
    SSY=y.var() * len(y),
)


======== Epoch 1 / 32 ========
Training...
  Batch    40  of    704.    Elapsed: 0:00:00.
  Batch    80  of    704.    Elapsed: 0:00:00.
  Batch   120  of    704.    Elapsed: 0:00:00.
  Batch   160  of    704.    Elapsed: 0:00:00.
  Batch   200  of    704.    Elapsed: 0:00:00.
  Batch   240  of    704.    Elapsed: 0:00:00.
  Batch   280  of    704.    Elapsed: 0:00:01.
  Batch   320  of    704.    Elapsed: 0:00:01.
  Batch   360  of    704.    Elapsed: 0:00:01.
  Batch   400  of    704.    Elapsed: 0:00:01.
  Batch   440  of    704.    Elapsed: 0:00:01.
  Batch   480  of    704.    Elapsed: 0:00:01.
  Batch   520  of    704.    Elapsed: 0:00:01.
  Batch   560  of    704.    Elapsed: 0:00:01.
  Batch   600  of    704.    Elapsed: 0:00:01.
  Batch   640  of    704.    Elapsed: 0:00:01.
  Batch   680  of    704.    Elapsed: 0:00:01.

  Average training loss: 512512.87
  Training epoch took: 0:00:01

Running Validation...
  Validation Loss: 48867.79
  Validation CI: +/-4326.17
  Validatio

  Batch   560  of    704.    Elapsed: 0:00:01.
  Batch   600  of    704.    Elapsed: 0:00:01.
  Batch   640  of    704.    Elapsed: 0:00:01.
  Batch   680  of    704.    Elapsed: 0:00:01.

  Average training loss: 14705.66
  Training epoch took: 0:00:01

Running Validation...
  Validation Loss: 14398.89
  Validation CI: +/-1785.70
  Validation SSE: 72687040.00
  Validation took: 0:00:00
  Validation R^2: 0.9969185590744019

======== Epoch 9 / 32 ========
Training...
  Batch    40  of    704.    Elapsed: 0:00:00.
  Batch    80  of    704.    Elapsed: 0:00:00.
  Batch   120  of    704.    Elapsed: 0:00:00.
  Batch   160  of    704.    Elapsed: 0:00:00.
  Batch   200  of    704.    Elapsed: 0:00:00.
  Batch   240  of    704.    Elapsed: 0:00:00.
  Batch   280  of    704.    Elapsed: 0:00:01.
  Batch   320  of    704.    Elapsed: 0:00:01.
  Batch   360  of    704.    Elapsed: 0:00:01.
  Batch   400  of    704.    Elapsed: 0:00:01.
  Batch   440  of    704.    Elapsed: 0:00:01.
  Batch   48

  Batch   280  of    704.    Elapsed: 0:00:01.
  Batch   320  of    704.    Elapsed: 0:00:01.
  Batch   360  of    704.    Elapsed: 0:00:01.
  Batch   400  of    704.    Elapsed: 0:00:01.
  Batch   440  of    704.    Elapsed: 0:00:01.
  Batch   480  of    704.    Elapsed: 0:00:01.
  Batch   520  of    704.    Elapsed: 0:00:01.
  Batch   560  of    704.    Elapsed: 0:00:01.
  Batch   600  of    704.    Elapsed: 0:00:01.
  Batch   640  of    704.    Elapsed: 0:00:01.
  Batch   680  of    704.    Elapsed: 0:00:01.

  Average training loss: 9088.23
  Training epoch took: 0:00:01

Running Validation...
  Validation Loss: 8918.44
  Validation CI: +/-1185.89
  Validation SSE: 45011800.00
  Validation took: 0:00:00
  Validation R^2: 0.9980918169021606

======== Epoch 17 / 32 ========
Training...
  Batch    40  of    704.    Elapsed: 0:00:00.
  Batch    80  of    704.    Elapsed: 0:00:00.
  Batch   120  of    704.    Elapsed: 0:00:00.
  Batch   160  of    704.    Elapsed: 0:00:00.
  Batch   200

  Batch    40  of    704.    Elapsed: 0:00:00.
  Batch    80  of    704.    Elapsed: 0:00:00.
  Batch   120  of    704.    Elapsed: 0:00:00.
  Batch   160  of    704.    Elapsed: 0:00:00.
  Batch   200  of    704.    Elapsed: 0:00:00.
  Batch   240  of    704.    Elapsed: 0:00:00.
  Batch   280  of    704.    Elapsed: 0:00:01.
  Batch   320  of    704.    Elapsed: 0:00:01.
  Batch   360  of    704.    Elapsed: 0:00:01.
  Batch   400  of    704.    Elapsed: 0:00:01.
  Batch   440  of    704.    Elapsed: 0:00:01.
  Batch   480  of    704.    Elapsed: 0:00:01.
  Batch   520  of    704.    Elapsed: 0:00:01.
  Batch   560  of    704.    Elapsed: 0:00:01.
  Batch   600  of    704.    Elapsed: 0:00:01.
  Batch   640  of    704.    Elapsed: 0:00:01.
  Batch   680  of    704.    Elapsed: 0:00:01.

  Average training loss: 6804.33
  Training epoch took: 0:00:01

Running Validation...
  Validation Loss: 6119.82
  Validation CI: +/-988.50
  Validation SSE: 30846316.00
  Validation took: 0:00:00
  

  Batch   640  of    704.    Elapsed: 0:00:01.
  Batch   680  of    704.    Elapsed: 0:00:01.

  Average training loss: 5540.51
  Training epoch took: 0:00:01

Running Validation...
  Validation Loss: 5732.84
  Validation CI: +/-815.43
  Validation SSE: 28748632.00
  Validation took: 0:00:00
  Validation R^2: 0.9987812638282776

======== Epoch 32 / 32 ========
Training...
  Batch    40  of    704.    Elapsed: 0:00:00.
  Batch    80  of    704.    Elapsed: 0:00:00.
  Batch   120  of    704.    Elapsed: 0:00:00.
  Batch   160  of    704.    Elapsed: 0:00:00.
  Batch   200  of    704.    Elapsed: 0:00:00.
  Batch   240  of    704.    Elapsed: 0:00:00.
  Batch   280  of    704.    Elapsed: 0:00:01.
  Batch   320  of    704.    Elapsed: 0:00:01.
  Batch   360  of    704.    Elapsed: 0:00:01.
  Batch   400  of    704.    Elapsed: 0:00:01.
  Batch   440  of    704.    Elapsed: 0:00:01.
  Batch   480  of    704.    Elapsed: 0:00:01.
  Batch   520  of    704.    Elapsed: 0:00:01.
  Batch   560 

In [26]:
torch.save(model.state_dict, "best_model.pt")